In [7]:
import json
import time
import pickle
import sys
import os

import zstandard as zstd
import pandas as pd
import numpy as np

from scipy.sparse import dok_matrix

#### Helpers

In [8]:
# Dictionnary mapping the video_id to the channel_id
vid_to_channels = pd.read_pickle("/dlabdata1/youtube_large/id_to_channel_mapping.pickle")

In [9]:
# Channels that are in set_crawler dataset and also in which the language is in english
with open('/dlabdata1/youtube_large/olam/channels_id_filtered.pickle', 'rb') as f:
    channels_id = pickle.load(f)
f.close()
channels_id = sorted(channels_id)
# Dictionnary to map the channel id to an integer corresponding to the column/row of the sparse matrix.
channel_dict = {}
for ind, channel_id in enumerate(channels_id):
    channel_dict[channel_id] = ind
channels_id = set(channels_id)

In [10]:
'''
Using Zreader enables to read the .zst files line by line
'''
class Zreader:

    def __init__(self, file, chunk_size=16384):
        '''Init method'''
        self.fh = open(file,'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor()
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''

    def readlines(self):
        '''Generator method that creates an iterator for each line of JSON'''
        while True:
            chunk = self.reader.read(self.chunk_size).decode("utf-8", errors="ignore")
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]


#### Create the bipartite graph between the users and the channels using sparse matrices

In [11]:
def check_directory(dir_1):
    
    if not os.path.exists(dir_1): 
        os.makedirs(dir_1)

In [16]:
haha = [3, 1, 2, 3]

In [17]:
haha.count(3)

2

In [7]:
def create_edges(graph_dict, user, user_channels, nb_users):
    set_user_channels = set(user_channels)
    max_freq = 0
    for channel in set_user_channels:
        channel_freq = user_channels.count(channel)
        if channel_freq > max_freq:
            max_freq = channel_freq
    for channel in set_user_channels:
        tf = user_channels.count(channel) / max_freq
        idf = nb_users / channels_idx_to_nb_comments[channel]
        graph_dict[(channel, user)] = tf * idf

In [46]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=160384)

# PARAMETERS
# Dictionnary containing edges of the bipartite graph
graph_dict = {}
#user_set = {}
idx = 1
user_idx = 0
nb = 1

dir_1 = '/dlabdata1/youtube_large/jouven/users/'
check_directory(dir_1)

# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if idx > 1:
        if len(line_split) == 9:
            if vid_to_channels.get(line_split[2]) in channels_id:
                if line_split[0] != user:
                    #user_set[line_split[0]] = user_idx
                    user_idx += 1
     
    idx += 1
    user = line_split[0]
    if user_idx >= 24989140 and user_idx <= 24989280:
        #with open("/dlabdata1/youtube_large/jouven/users/users_to_idx_"+str(2)+".pkl",'wb') as f:
        #    pickle.dump(user_set, f)
        #f.close()
        #user_set = {}
        print(line_split)
        nb += 1
    
    if user_idx == 24989280:
        break
        
#with open("/dlabdata1/youtube_large/jouven/users/users_to_idx_"+str(nb)+".pkl",'wb') as f:
#    pickle.dump(user_set, f)
#f.close()


['UCP6cpvd7qcvHDM6l6Tl0A5g', 'UgyhZ7QrDamn2Q8FyxR4AaABAg', 'euUiIjXA-zc', '', '1571478278', '0', '0', 'james h', '[{text:80$ to fill a tank? Welcome America to the fuel prices the rest of he world has had for last decade.}]']
['UCuCcuao1N8YfEbbz1iR6gtw', 'UggW7E9SYnIrXHgCoAEC.85jeJBR24hh87rzGoOOllB', 'rzxnEW1YJMM', 'UggW7E9SYnIrXHgCoAEC', '1571637712', '0', '0', 'Mehdi Mad', '[{text:+Hamid Bekhtiar nnkik}]']
['UCuCcuao1N8YfEbbz1iR6gtw', 'Ugh2sDEhyVTBWHgCoAEC', 'd9hg-7f_yMg', '', '1571637690', '0', '0', 'Mehdi Mad', '[{text:p}]']
['UCr5U5LhWRBf44olNv7YzlrQ', 'Ughk4D52XvZU9ngCoAEC', 'W-w3WfgpcGg', '', '1571618601', '0', '0', 'Caroline Campos', '[{text:Esse video e um dos melhores.}', '{text:\\n}]']
['UCBJuQ9D1G4xXdGEeAZ5jwqQ', 'UgyZ4sFFbvDnNOZ507V4AaABAg.8_ZRgT1m-Yr8_hDvR3cP79', 'ws00k_lIQ9U', 'UgyZ4sFFbvDnNOZ507V4AaABAg', '1571461277', '0', '0', 'Alejandra Jimenez', '[{text:Te amo}]']
['UCmSkiw2zJfXqCBckBk_8FOw', 'Ugg1N9Ya7Yrus3gCoAEC', 'drglk3XJA9o', '', '1571652145', '0', '2', 'Alicia

['UCEq6VtW_8UXnumP9biyGknA', 'Ugygcu3Pl_KuJOzy7YZ4AaABAg', 'nxgSXM8NUls', '', '1570563831', '0', '0', 'Invader Grimm', "[{text:Oh boy I hope they don't mess this one up pls don't if you do im going to be in Range it can be a good Movie but by the looks of it its kinda to late but ok lets see if this one makes you cry at night maybe not slender man is kinda not that scary no more what am I even Talking About who knows if this will be good or bad}]"]
['UCEq6VtW_8UXnumP9biyGknA', 'Ugygw-1VrzmMYMCNK8t4AaABAg', 'GcXNAwXfhy0', '', '1570773787', '0', '0', 'Invader Grimm', "[{text:What if the baryonyx as Rabies that's maybe why its trying to bite them or eat them}]"]
['UCEq6VtW_8UXnumP9biyGknA', 'UgykAPf0ia3tZs9jIo54AaABAg', 'iWmSToylW6M', '', '1571729725', '0', '0', 'Invader Grimm', '[{text:I hope that every cat got a new family good job your all Great for helping dose poor Cats}]']
['UCEq6VtW_8UXnumP9biyGknA', 'Ugyks-SwzpzxfYlu0bJ4AaABAg', '0-7iXyYS0uw', '', '1571761472', '0', '0', 'Invader 

In [32]:
with open("/dlabdata1/youtube_large/jouven/users/users_tab.pkl",'rb') as f:
    user_set2 = pickle.load(f)
f.close()

In [33]:
last_val = 0
for key, val in user_set.items():
    if val != last_val+1:
        print('key ' + str(key) + 'val ' + str(val))
    last_val = val

key UCTvmM0NQyzNty5gus9Nmoegval 0
key UCvIe0MFxuC7e3FR1Xyrq-3Qval 24989263


In [43]:
user_set2.count('UCXmTHnPrTnlHDwbzKeWaryQ')

2

In [41]:
for ind, val in enumerate(user_set2):
    if val == 'UCvIe0MFxuC7e3FR1Xyrq-3Q':
        print(ind)

24989145
24989263


In [45]:
for ind, val in enumerate(user_set2):
    if val == 'UCXmTHnPrTnlHDwbzKeWaryQ':
        print(ind)

24989146
24989264


In [44]:
user_set2[24989147]

'UCm-hAqNBQn1Aroo95aMYPjQ'

In [26]:
with open("/dlabdata1/youtube_large/jouven/users/users_to_idx_"+str(1)+".pkl",'rb') as f:
    user_set2 = pickle.load(f)
f.close()

In [27]:
user_set2

{}